# This is the tutorials of using PyABSA for aspect term extraction
Drafted for v2.0 and higher versions. Note there are many breaking changes in v2.0, so you do not need to upgrade to v2.0 and higher versions if you are using code, API, checkpoints, datasets or anything from v1.0. Let's begin the introduction.

In [1]:
!pip install pyabsa -U
from pyabsa import AspectTermExtraction as ATEPC

C:\Users\chuan\miniconda3\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
C:\Users\chuan\AppData\Roaming\Python\Python39\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


[2023-02-22 16:30:34] (2.0.29a0) PyABSA(2.0.29a0): If you find any problems, please report them on GitHub. Thanks!
The v2.x versions are not compatible with Google Colab. Please downgrade to 1.16.27.


C:\Users\chuan\miniconda3\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


# ATEPCModelList
There are three types of APC models for aspect term extraction, which are based on the local context focus mechanism
Notice: when you select to use a model, please make sure to carefully manage the configurations, e.g., for glove-based models, you need to set hidden dim and embed_dim manually.
We already provide some pre-defined configurations. Refer to the source code if you have any question
e.g.,

In [2]:
config = ATEPC.ATEPCConfigManager.get_atepc_config_english()  # this config contains 'pretrained_bert', it is based on pretrained models
config.model = ATEPC.ATEPCModelList.FAST_LCF_ATEPC  # improved version of LCF-ATEPC

In [ ]:
![Screenshot.png](Screenshot.png)


# ATEPCDatasetList
There are the [datasets](https://github.com/yangheng95/ABSADatasets) from publication or third-party contribution. There dataset can be downloaded and processed automatically.
In pyabsa, you can pass a set of datasets to train a model.
e.g., for using integrated datasets:


In [4]:
from pyabsa import DatasetItem

dataset = ATEPC.ATEPCDatasetList.Restaurant14
# now the dataset is a DatasetItem object, which has a name and a list of subdatasets
# e.g., SemEval dataset contains Laptop14, Restaurant14, Restaurant16 datasets

You can use your own dataset provided that it is formatted according to [ABSADatasets](https://github.com/yangheng95/ABSADatasets#important-rename-your-dataset-filename-before-use-it-in-pyabsa)

In [5]:
# Put your dataset into integrated_datasets folder, it this folder does not exist, you need to call:
from pyabsa import download_all_available_datasets

download_all_available_datasets()

[2023-02-14 06:08:58] (2.0.28a0) Datasets already exist in /home/yangheng/pyabsa/examples-v2/aspect_term_extraction/integrated_datasets, skip download


to pass datasets to PyABSA trainers, you can

In [ ]:
my_dataset = DatasetItem("my_dataset", ["my_dataset1", "my_dataset2"])
# my_dataset1 and my_dataset2 are the dataset folders. In there folders, the train dataset is necessary

# Training
Let's prepare to train

In [5]:
from pyabsa import ModelSaveOption, DeviceTypeOption
import warnings

warnings.filterwarnings("ignore")

config.batch_size = 16
config.patience = 2
config.log_step = -1
config.seed = [1, 2, 3]
config.verbose = False  # If verbose == True, PyABSA will output the model strcture and seversal processed data examples
config.notice = (
    "This is an training example for aspect term extraction"  # for memos usage
)

trainer = ATEPC.ATEPCTrainer(
    config=config,
    dataset=dataset,
    from_checkpoint="english",  # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,  # use cuda if available
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,  # save state dict only instead of the whole model
    load_aug=False,  # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
)

[2023-02-14 07:40:24] (2.0.28a0) Set Model Device: cuda:1
[2023-02-14 07:40:24] (2.0.28a0) Device Name: NVIDIA GeForce RTX 3090
2023-02-14 07:40:25,512 INFO: PyABSA version: 2.0.28a0
2023-02-14 07:40:25,515 INFO: Transformers version: 4.25.1
2023-02-14 07:40:25,517 INFO: Torch version: 2.0.0.dev20221210+cu117+cuda11.7
2023-02-14 07:40:25,518 INFO: Device: NVIDIA GeForce RTX 3090
2023-02-14 07:40:25,527 INFO: Searching dataset 114.Restaurant14 in local disk
2023-02-14 07:40:25,559 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.
2023-02-14 07:40:25,560 INFO: Please use a new folder to perform new text augment if the former augment in integrated_datasets/atepc_datasets/110.SemEval/114.restaurant14 errored unexpectedly


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
convert examples to features:  14%|█▎        | 492/3608 [00:00<00:01, 1649.17it/s]

2023-02-14 07:40:27,702 WARNING: AspectTooLongWarning -> <aspect: fried mini buns with the condensed milk and the assorted fruits on beancurd> is too long, <text: The waitress remembers me and is very friendly , she knows what my regular is and that ' s the fried mini buns with the condensed milk and the assorted fruits on beancurd .>, <polarity: Positive>


convert examples to features:  47%|████▋     | 1695/3608 [00:01<00:01, 1695.67it/s]

2023-02-14 07:40:28,366 WARNING: AspectTooLongWarning -> <aspect: salad with perfectly marinated cucumbers and tomatoes with lots of shrimp and basil> is too long, <text: I ate clams oreganta and spectacular salad with perfectly marinated cucumbers and tomatoes with lots of shrimp and basil .>, <polarity: Positive>


...

2023-02-14 07:48:58,202 INFO: Dataset Label Details: {'Positive': 726, 'Neutral': 196, 'Negative': 196, 'Sum': 1118}



Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2023-02-14 07:48:59,887 INFO: Save cache dataset to fast_lcf_atepc.Restaurant14.dataset.70b14ee4d789e060334c3eea7126a5e56f7f9b88b04805b0c961907099a7214c.cache
2023-02-14 07:49:00,052 INFO: cuda memory allocated:776236032
2023-02-14 07:49:00,052 INFO: ABSADatasetsVersion:None	-->	Calling Count:0
2023-02-14 07:49:00,053 INFO: IOB_label_to_index:{'B-ASP': 1, 'I-ASP': 2, 'O': 3, '[CLS]': 4, '[SEP]': 5}	-->	Calling Count:3
2023-02-14 07:49:00,053 INFO: MV:<metric_visualizer.metric_visualizer.MetricVisualizer object at 0x7f56f8eac2b0>	-->	Calling Count:10
2023-02-14 07:49:00,053 INFO: PyABSAVersion:2.0.28a0	-->	Calling Count:1
2023-02-14 07:49:00,053 INFO: SRD:3	-->	Calling Count:28332
2023-02-14 07:49:00,054 INFO: TorchVersion:2.0.0.dev20221210+cu117+cuda11.7	-->	Calling Count:1
2023-02-14 07:49:00,054 INFO: TransformersVersion:4.25.1	-->	Calling Count:1
2023-02-14 07:49:00,054 INFO: auto_device:True	-->	Calling Count:1814
2023-02-14 07:49:00,055 INFO: batch_size:16	-->	Calling Count:16
202

Epoch:  0| loss_apc:0.0099 | loss_ate:0.0139 |: 100%|██████████| 226/226 [01:02<00:00,  3.59it/s,  APC_ACC: 85.96(max:87.21) | APC_F1: 78.04(max:81.65) | ATE_F1: 82.87(max:84.58)]
Epoch:  1| loss_apc:0.0429 | loss_ate:0.0058 |: 100%|██████████| 226/226 [01:03<00:00,  3.58it/s,  APC_ACC: 88.46(max:88.46) | APC_F1: 82.98(max:82.98) | ATE_F1: 83.60(max:84.79)]
Epoch:  2| loss_apc:0.0007 | loss_ate:0.0598 |: 100%|██████████| 226/226 [01:02<00:00,  3.60it/s,  APC_ACC: 87.57(max:88.46) | APC_F1: 81.32(max:82.98) | ATE_F1: 85.72(max:85.72)]
Epoch:  3| loss_apc:0.0063 | loss_ate:0.0017 |: 100%|██████████| 226/226 [01:02<00:00,  3.63it/s,  APC_ACC: 88.19(max:88.46) | APC_F1: 82.75(max:82.98) | ATE_F1: 84.08(max:85.72)]


2023-02-14 07:53:15,032 INFO: 
--------------------------------------------------------------------------- Metric Visualizer ---------------------------------------------------------------------------
╒════════════════════════════════╤═══════════════════════════════════════════════════════╤═══════════════════════╤═══════════╤══════════╤═══════╤═══════╤═══════╤═══════╕
│ Metric                         │ Trial                                                 │ Values                │  Average  │  Median  │  Std  │  IQR  │  Min  │  Max  │
╞════════════════════════════════╪═══════════════════════════════════════════════════════╪═══════════════════════╪═══════════╪══════════╪═══════╪═══════╪═══════╪═══════╡
│ Max-APC-Test-Acc w/o Valid Set │ fast_lcf_atepc-Restaurant14-microsoft/deberta-v3-base │ [87.92, 88.28, 88.46] │   88.22   │  88.28   │ 0.22  │ 0.27  │ 87.92 │ 88.46 │
├────────────────────────────────┼───────────────────────────────────────────────────────┼───────────────────────┼─────

to load trained model for inference:

In [16]:
aspect_extractor = trainer.load_trained_model()
assert isinstance(aspect_extractor, ATEPC.AspectExtractor)

[2023-02-14 06:46:05] (2.0.28a0) Load aspect extractor from checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_87.3_apcf1_81.39_atef1_86.06
[2023-02-14 06:46:05] (2.0.28a0) config: checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_87.3_apcf1_81.39_atef1_86.06/fast_lcf_atepc.config
[2023-02-14 06:46:05] (2.0.28a0) state_dict: checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_87.3_apcf1_81.39_atef1_86.06/fast_lcf_atepc.state_dict
[2023-02-14 06:46:05] (2.0.28a0) model: None
[2023-02-14 06:46:05] (2.0.28a0) tokenizer: checkpoints/fast_lcf_atepc_Restaurant14_cdw_apcacc_87.3_apcf1_81.39_atef1_86.06/fast_lcf_atepc.tokenizer
[2023-02-14 06:46:06] (2.0.28a0) Set Model Device: cuda:1
[2023-02-14 06:46:06] (2.0.28a0) Device Name: NVIDIA GeForce RTX 3090


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

# Inference

## Use our checkpoints to initialize a AspectExtractor and SentimentClassifier

In [2]:
from pyabsa import available_checkpoints

ckpts = (
    available_checkpoints()
)  # This will show the available checkpoints and their detailed informantion
# find a suitable checkpoint and use the name:
aspect_extractor = ATEPC.AspectExtractor(
    checkpoint="english"
)  # here I use the english checkpoint which is trained on all English datasets in PyABSA

[2023-02-22 16:30:34] (2.0.29a0) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2023-02-22 16:30:35] (2.0.29a0) ********** Available ATEPC model checkpoints for Version:2.0.29a0 (this version) **********
[2023-02-22 16:30:35] (2.0.29a0) Downloading checkpoint:english 
[2023-02-22 16:30:35] (2.0.29a0) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets


[2023-02-22 16:30:41] (2.0.29a0) Find zipped checkpoint: ./checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip, unzipping


[2023-02-22 16:30:45] (2.0.29a0) Done.
[2023-02-22 16:30:45] (2.0.29a0) If the auto-downloading failed, please download it via browser: https://huggingface.co/spaces/yangheng/PyABSA/resolve/main/checkpoints/English/ATEPC/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip 
[2023-02-22 16:30:45] (2.0.29a0) Load aspect extractor from ./checkpoints\ATEPC_ENGLISH_CHECKPOINT
[2023-02-22 16:30:45] (2.0.29a0) config: ./checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.config
[2023-02-22 16:30:45] (2.0.29a0) state_dict: ./checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.state_dict
[2023-02-22 16:30:45] (2.0.29a0) model: None
[2023-02-22 16:30:45] (2.0.29a0) tokenizer: ./checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.tokenizer
[2023-02-22 16:30:45] (2.0.29a0) Set Model Device: cud

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have b

## Simple Prediction

In [10]:
atepc_examples = [
    "But the staff was so nice to us .",
    "But the staff was so horrible to us .",
    r"Not only was the food outstanding , but the little ` perks \' were great .",
    "It took half an hour to get our check , which was perfect since we could sit , have drinks and talk !",
    "It was pleasantly uncrowded , the service was delightful , the garden adorable , "
    "the food -LRB- from appetizers to entrees -RRB- was delectable .",
    "How pretentious and inappropriate for MJ Grill to claim that it provides power lunch and dinners !",
]
# predict interface accepts a list of example or a single example
for ex in atepc_examples:
    result = aspect_extractor.predict(
        example=ex,
        print_result=True,
        ignore_error=True,  # ignore an invalid example, if it is False, invalid examples will raise Exceptions
        eval_batch_size=32,
    )

D:\Works\PyABSA\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:419: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = [float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()]


The results of aspect term extraction have been saved in D:\Works\PyABSA\examples-v2\aspect_term_extraction\atepc_inference.result.json
Example 0: But the <staff:Positive Confidence:0.999491810798645> was so nice to us .
The results of aspect term extraction have been saved in D:\Works\PyABSA\examples-v2\aspect_term_extraction\atepc_inference.result.json
Example 0: But the <staff:Negative Confidence:0.9985008239746094> was so horrible to us .
The results of aspect term extraction have been saved in D:\Works\PyABSA\examples-v2\aspect_term_extraction\atepc_inference.result.json
Example 0: Not only was the <food:Positive Confidence:0.9992227554321289> outstanding , but the little ` <perks:Positive Confidence:0.9973457455635071> \ ' were great .
The results of aspect term extraction have been saved in D:\Works\PyABSA\examples-v2\aspect_term_extraction\atepc_inference.result.json
Example 0: It took half an hour to get our <check:Neutral Confidence:0.9945604205131531> , which was perfect sin

## Batch Inference

In [4]:
aspect_extractor.batch_predict(
    target_file=ATEPC.ATEPCDatasetList.Restaurant16,
    print_result=True,
    save_result=False,
    ignore_error=True,
    eval_batch_size=32,
)

[2023-02-22 16:32:25] (2.0.29a0) Try to load 116.Restaurant16 dataset from local disk
[2023-02-22 16:32:25] (2.0.29a0) loading: integrated_datasets\apc_datasets\110.SemEval\116.restaurant16\restaurant_test.raw.inference


preparing apc inference dataloader: 100%|██████████| 710/710 [00:01<00:00, 384.63it/s]
C:\Users\chuan\OneDrive - University of Exeter\AIProjects\PyABSA\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:567: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:253.)
  lcf_cdm_vec = torch.tensor(
classifying aspect sentiments:   0%|          | 0/23 [00:00<?, ?it/s]C:\Users\chuan\OneDrive - University of Exeter\AIProjects\PyABSA\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:648: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()
classifying aspect sentiments: 100%|█

[2023-02-22 16:32:59] (2.0.29a0) Example 0: serves really good <sushi:Positive Confidence:0.9919> .
[2023-02-22 16:32:59] (2.0.29a0) Example 1: not the biggest <portions:Negative Confidence:0.9795> but adequate .
[2023-02-22 16:32:59] (2.0.29a0) Example 2: green tea creme brulee is a must !
[2023-02-22 16:32:59] (2.0.29a0) Example 3:   – i ca n ' t say enough about <this:Positive Confidence:0.9955> place .
[2023-02-22 16:32:59] (2.0.29a0) Example 4: it has great <sushi:Positive Confidence:0.9995> and even better <service:Positive Confidence:0.9995> .
[2023-02-22 16:32:59] (2.0.29a0) Example 5: the entire <staff:Positive Confidence:0.9995> was extremely accomodating and tended to my every need .
[2023-02-22 16:32:59] (2.0.29a0) Example 6: i ' ve been to this <restaurant:Positive Confidence:0.9987> over a dozen times with no complaints to date .
[2023-02-22 16:32:59] (2.0.29a0) Example 7: the <owner:Negative Confidence:0.9985> is belligerent to guests that have a complaint .
[2023-02-22 

[{'sentence': 'serves really good sushi .',
  'IOB': ['B-ASP', 'O', 'O', 'B-ASP', 'O'],
  'tokens': ['serves', 'really', 'good', 'sushi', '.'],
  'aspect': ['serves', 'sushi'],
  'position': [[1], [4]],
  'sentiment': ['Neutral', 'Positive'],
  'probs': [[0.0004788661899510771, 0.6750665307044983, 0.3244546353816986],
   [0.0002311569405719638, 0.007874456234276295, 0.9918943047523499]],
  'confidence': [0.6751, 0.9919]},
 {'sentence': 'not the biggest portions but adequate .',
  'IOB': ['O', 'O', 'O', 'B-ASP', 'O', 'O', 'O'],
  'tokens': ['not', 'the', 'biggest', 'portions', 'but', 'adequate', '.'],
  'aspect': ['portions'],
  'position': [[4]],
  'sentiment': ['Negative'],
  'probs': [[0.979500949382782, 0.00579943647608161, 0.014699668623507023]],
  'confidence': [0.9795]},
 {'sentence': 'green tea creme brulee is a must !',
  'IOB': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
  'tokens': ['green', 'tea', 'creme', 'brulee', 'is', 'a', 'must', '!'],
  'aspect': [],
  'position': [],
  

# Annotate your own datasets based on PyABSA
[Auto-Annotation](https://github.com/yangheng95/ABSADatasets#auto-annoate-your-datasets-via-pyabsa)  # available for v1.0 currently
[Manually-Annotation](https://github.com/yangheng95/ABSADatasets/tree/v1.2/DPT)

# Deploy a ATEPC demo
Here is a simple web-based demo for aspect term extraction


In [2]:
import os
import random
import gradio as gr
import pandas as pd
import requests

from pyabsa import (
    download_all_available_datasets,
    AspectTermExtraction as ATEPC,
    TaskCodeOption,
)
from pyabsa.utils.data_utils.dataset_manager import detect_infer_dataset

download_all_available_datasets()

dataset_items = {dataset.name: dataset for dataset in ATEPC.ATEPCDatasetList()}


def get_example(dataset):
    task = TaskCodeOption.Aspect_Polarity_Classification
    dataset_file = detect_infer_dataset(dataset_items[dataset], task)

    for fname in dataset_file:
        lines = []
        if isinstance(fname, str):
            fname = [fname]

        for f in fname:
            print("loading: {}".format(f))
            fin = open(f, "r", encoding="utf-8")
            lines.extend(fin.readlines())
            fin.close()
        for i in range(len(lines)):
            lines[i] = (
                lines[i][: lines[i].find("$LABEL$")]
                .replace("[B-ASP]", "")
                .replace("[E-ASP]", "")
                .strip()
            )
        return sorted(set(lines), key=lines.index)


dataset_dict = {
    dataset.name: get_example(dataset.name) for dataset in ATEPC.ATEPCDatasetList()
}
aspect_extractor = ATEPC.AspectExtractor(checkpoint="multilingual")


def perform_inference(text, dataset):
    if not text:
        text = dataset_dict[dataset][random.randint(0, len(dataset_dict[dataset]) - 1)]

    result = aspect_extractor.predict(text, pred_sentiment=True)

    result = pd.DataFrame(
        {
            "aspect": result["aspect"],
            "sentiment": result["sentiment"],
            # 'probability': result[0]['probs'],
            "confidence": [round(x, 4) for x in result["confidence"]],
            "position": result["position"],
        }
    )
    return result, "{}".format(text)


demo = gr.Blocks()

with demo:
    gr.Markdown(
        "# <p align='center'>Multilingual Aspect-based Sentiment Analysis !</p>"
    )
    output_dfs = []
    with gr.Row():
        with gr.Column():
            input_sentence = gr.Textbox(
                placeholder="Leave this box blank and choose a dataset will give you a random example...",
                label="Example:",
            )
            gr.Markdown(
                "You can find the datasets at [github.com/yangheng95/ABSADatasets](https://github.com/yangheng95/ABSADatasets/tree/v1.2/datasets/text_classification)"
            )
            dataset_ids = gr.Radio(
                choices=[dataset.name for dataset in ATEPC.ATEPCDatasetList()[:-1]],
                value="Laptop14",
                label="Datasets",
            )
            inference_button = gr.Button("Let's go!")
            gr.Markdown(
                "There is a [demo](https://huggingface.co/spaces/yangheng/PyABSA-ATEPC-Chinese) specialized for the Chinese langauge"
            )
            gr.Markdown(
                "This demo support many other language as well, you can try and explore the results of other languages by yourself."
            )

        with gr.Column():
            output_text = gr.TextArea(label="Example:")
            output_df = gr.DataFrame(label="Prediction Results:")
            output_dfs.append(output_df)

        inference_button.click(
            fn=perform_inference,
            inputs=[input_sentence, dataset_ids],
            outputs=[output_df, output_text],
        )

demo.launch()

[2023-02-14 00:56:56] (2.0.28) Datasets already exist in C:\Users\chuan\OneDrive - University of Exeter\AIProjects\PyABSA\examples-v2\aspect_term_extraction\integrated_datasets, skip download
[2023-02-14 00:56:56] (2.0.28) Try to load 113.Laptop14 dataset from local disk
loading: integrated_datasets\apc_datasets\110.SemEval\113.laptop14\Laptops_Test_Gold.xml.seg.inference
[2023-02-14 00:56:56] (2.0.28) Try to load 114.Restaurant14 dataset from local disk
loading: integrated_datasets\apc_datasets\110.SemEval\114.restaurant14\Restaurants_Test_Gold.xml.seg.inference
[2023-02-14 00:56:56] (2.0.28) Try to load 111.ARTS_Laptop14 dataset from local disk
loading: integrated_datasets\apc_datasets\110.SemEval\111.arts_laptop14\laptop_arts_test.dat.inference
[2023-02-14 00:56:56] (2.0.28) Try to load 112.ARTS_Restaurant14 dataset from local disk
loading: integrated_datasets\apc_datasets\110.SemEval\112.arts_restaurant14\rest_arts_test.dat.inference
[2023-02-14 00:56:56] (2.0.28) Try to load 115.R

Some weights of the model checkpoint at microsoft/mdeberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have 

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


C:\Users\chuan\AppData\Roaming\Python\Python310\site-packages\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:647: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()


[2023-02-14 00:57:14] (2.0.28) The results of aspect term extraction have been saved in C:\Users\chuan\OneDrive - University of Exeter\AIProjects\PyABSA\examples-v2\aspect_term_extraction\Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-02-14 00:57:14] (2.0.28) Example 0: The only solution is to turn the <brightness:Neutral Confidence:0.8198688626289368> down , etc . .


C:\Users\chuan\AppData\Roaming\Python\Python310\site-packages\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:647: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  float(x) for x in F.softmax(i_apc_logits).cpu().numpy().tolist()


[2023-02-14 00:57:34] (2.0.28) The results of aspect term extraction have been saved in C:\Users\chuan\OneDrive - University of Exeter\AIProjects\PyABSA\examples-v2\aspect_term_extraction\Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2023-02-14 00:57:34] (2.0.28) Example 0: In fact I still use many Legacy <programs:Neutral Confidence:0.9923437237739563> - LRB - <Appleworks:Neutral Confidence:0.9929322004318237> , <FileMaker Pro:Neutral Confidence:0.9934661984443665> , <Quicken:Neutral Confidence:0.9928175210952759> , <Photoshop:Neutral Confidence:0.9908045530319214> etc - RRB - !
